#### Translate geotiff 16bit to grayscale 8bit, as tensorflow, can't handle 16bit images and 16bit rgb conversion to 8bit losses too much info ([Why training your CNN](https://towardsdatascience.com/why-training-your-cnn-with-16-bit-images-isnt-working-e91da350c49d))


In [45]:
output_png = '../out/out_8bit/out_kaunas_sentinel_grayscale.png'
input_geotiff = '../out/out_kaunas_sentinel.tif'
training_data_images_output = '/media/agmis/mokslams/copernicus/the_third_eye/out/training_data/images'

In [46]:
from osgeo import gdal

scale = '-scale min_val max_val'
options_list = [
    '-ot Byte',
    '-of JPEG',
    scale
] 
options_string = " ".join(options_list)

gdal.Translate(output_png,
               input_geotiff,
               options=options_string)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f2d6a3c0ab0> >

Prepare annotations

In [55]:
output_annotations_png = '../masks/norm_img.png'
input_annotations_geotiff = '../masks/polygon_kauno_r_m.tif'
training_data_annotations_output = '/media/agmis/mokslams/copernicus/the_third_eye/out/training_data/annotations'

In [54]:
scale = '-scale min_val max_val'
options_list = [
    '-ot Byte',
    '-of JPEG',
    scale
] 
options_string = " ".join(options_list)

gdal.Translate(output_annotations_png,
               input_annotations_geotiff,
               options=options_string)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f2d6a296c30> >

#### Cut grayscale image to pieces

In [57]:
from PIL import Image
def crop(path, input, annotations=False):
    height = 255
    width = 255
    im = Image.open(input)
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            try:
                if annotations:
                    if a.getbbox():
                        a.save(os.path.join(path, f'{i}{j}.png'))
                else:
                    a.save(os.path.join(path, f'{i}{j}.png'))
            except Exception as e:
                print(e)


In [49]:
crop(training_data_images_output, output_png)

In [58]:
crop(training_data_annotations_output, output_annotations_png, True)

#### Transfer images with annotations

In [61]:
import os
import shutil
training_data_images_that_have_annotations_output = '/media/agmis/mokslams/copernicus/the_third_eye/out/training_data/images_that_have_annotations'
for file in os.listdir(training_data_annotations_output):
    shutil.copy(os.path.join(training_data_images_output, file), os.path.join(training_data_images_that_have_annotations_output, file))